# 参数管理

在选择了架构并设置了超参数后，我们就进入了训练阶段。
此时，我们的目标是找到使损失函数最小化的模型参数值。
经过训练后，我们将需要使用这些参数来做出未来的预测。
此外，有时我们希望提取参数，以便在其他环境中复用它们，
将模型保存下来，以便它可以在其他软件中执行，
或者为了获得科学的理解而进行检查。

之前的介绍中，我们只依靠深度学习框架来完成训练的工作，
而忽略了操作参数的具体细节。
本节，我们将介绍以下内容：

* 访问参数，用于调试、诊断和可视化；
* 参数初始化；
* 在不同模型组件间共享参数。

(**我们首先看一下具有单隐藏层的多层感知机。**)


In [2]:
import torch
from torch import nn

net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
X = torch.rand(size=(2, 4))
net(X)

tensor([[-0.2049],
        [-0.1654]], grad_fn=<AddmmBackward0>)

## [**参数访问**]

我们从已有模型中访问参数。
当通过`Sequential`类定义模型时，
我们可以通过索引来访问模型的任意层。
这就像模型是一个列表一样，每层的参数都在其属性中。
如下所示，我们可以检查第二个全连接层的参数。


In [3]:
print(net[2].state_dict())

OrderedDict([('weight', tensor([[-0.0454,  0.0922, -0.1534,  0.1332,  0.1545, -0.3042, -0.0138, -0.1680]])), ('bias', tensor([-0.3173]))])


In [4]:
print(dir(net[0]))

['T_destination', '__annotations__', '__call__', '__class__', '__constants__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_apply', '_backward_hooks', '_backward_pre_hooks', '_buffers', '_call_impl', '_compiled_call_impl', '_forward_hooks', '_forward_hooks_always_called', '_forward_hooks_with_kwargs', '_forward_pre_hooks', '_forward_pre_hooks_with_kwargs', '_get_backward_hooks', '_get_backward_pre_hooks', '_get_name', '_is_full_backward_hook', '_load_from_state_dict', '_load_state_dict_post_hooks', '_load_state_dict_pre_hooks', '_maybe_warn_non_full_backward_hook', '_modules', '_named_members', '_non_persistent_buffers_set', '_parameters', '_register_load_s

In [5]:
net[0]._parameters

{'weight': Parameter containing:
 tensor([[-0.2141, -0.4989,  0.2585,  0.4953],
         [ 0.0714, -0.4654,  0.2976,  0.4044],
         [-0.4842,  0.4036,  0.2129, -0.1694],
         [ 0.0931,  0.2491,  0.1665, -0.4766],
         [ 0.3988,  0.3981, -0.3291,  0.4456],
         [ 0.1332, -0.2593,  0.3112, -0.2198],
         [ 0.0101, -0.1279,  0.0331, -0.3786],
         [ 0.3397,  0.3609,  0.3108, -0.3490]], requires_grad=True),
 'bias': Parameter containing:
 tensor([ 0.3337,  0.3039, -0.3718,  0.3358,  0.2986,  0.0069, -0.1085,  0.2482],
        requires_grad=True)}

In [8]:
net[0].weight.data

tensor([[-0.2141, -0.4989,  0.2585,  0.4953],
        [ 0.0714, -0.4654,  0.2976,  0.4044],
        [-0.4842,  0.4036,  0.2129, -0.1694],
        [ 0.0931,  0.2491,  0.1665, -0.4766],
        [ 0.3988,  0.3981, -0.3291,  0.4456],
        [ 0.1332, -0.2593,  0.3112, -0.2198],
        [ 0.0101, -0.1279,  0.0331, -0.3786],
        [ 0.3397,  0.3609,  0.3108, -0.3490]])

输出的结果告诉我们一些重要的事情：
首先，这个全连接层包含两个参数，分别是该层的权重和偏置。
两者都存储为单精度浮点数（float32）。
注意，参数名称允许唯一标识每个参数，即使在包含数百个层的网络中也是如此。

### [**目标参数**]

注意，每个参数都表示为参数类的一个实例。
要对参数执行任何操作，首先我们需要访问底层的数值。
有几种方法可以做到这一点。有些比较简单，而另一些则比较通用。
下面的代码从第二个全连接层（即第三个神经网络层）提取偏置，
提取后返回的是一个参数类实例，并进一步访问该参数的值。


In [9]:
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([-0.3173], requires_grad=True)
tensor([-0.3173])


参数是复合的对象，包含值、梯度和额外信息。
这就是我们需要显式参数值的原因。
除了值之外，我们还可以访问每个参数的梯度。
在上面这个网络中，由于我们还没有调用反向传播，所以参数的梯度处于初始状态。


In [ ]:
net[2].weight.grad == None

True

### [**一次性访问所有参数**]

当我们需要对所有参数执行操作时，逐个访问它们可能会很麻烦。
当我们处理更复杂的块（例如，嵌套块）时，情况可能会变得特别复杂，
因为我们需要递归整个树来提取每个子块的参数。
下面，我们将通过演示来比较访问第一个全连接层的参数和访问所有层。


In [11]:
print(*[(name, param.shape) for name, param in net[0].named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))


In [13]:
print(*[(name, param.shape) for name, param in net.named_parameters()])

('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


这为我们提供了另一种访问网络参数的方式，如下所示。


In [ ]:
net.state_dict()['2.bias'].data

tensor([0.0887])

### [**从嵌套块收集参数**]

让我们看看，如果我们将多个块相互嵌套，参数命名约定是如何工作的。
我们首先定义一个生成块的函数（可以说是“块工厂”），然后将这些块组合到更大的块中。


In [14]:
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                         nn.Linear(8, 4), nn.ReLU())

def block2():
    net = nn.Sequential()
    for i in range(4):
        # 在这里嵌套
        net.add_module(f'block {i}', block1())
    return net

rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
rgnet(X)

tensor([[-0.1036],
        [-0.1037]], grad_fn=<AddmmBackward0>)

[**设计了网络后，我们看看它是如何工作的。**]


In [ ]:
print(rgnet)

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


因为层是分层嵌套的，所以我们也可以像通过嵌套列表索引一样访问它们。
下面，我们访问第一个主要的块中、第二个子块的第一层的偏置项。


In [15]:
rgnet[0][1][0].bias.data

tensor([ 0.1290, -0.2120, -0.0614,  0.0734, -0.1861, -0.2720, -0.3129,  0.1594])

In [18]:
rgnet[0][1][0].weight.data

tensor([[ 0.0967,  0.3918,  0.1014, -0.0013],
        [-0.3126,  0.4457, -0.1172,  0.4543],
        [ 0.1350,  0.4236,  0.1884,  0.4641],
        [-0.4602,  0.3499,  0.2631,  0.4761],
        [ 0.0301, -0.2835, -0.0878, -0.0179],
        [ 0.2013,  0.4277, -0.3432, -0.0019],
        [ 0.1341,  0.1906,  0.4755, -0.1735],
        [ 0.1281, -0.4298,  0.2946, -0.1972]])

## 参数初始化

知道了如何访问参数后，现在我们看看如何正确地初始化参数。
我们在 :numref:`sec_numerical_stability`中讨论了良好初始化的必要性。
深度学习框架提供默认随机初始化，
也允许我们创建自定义初始化方法，
满足我们通过其他规则实现初始化权重。


默认情况下，PyTorch会根据一个范围均匀地初始化权重和偏置矩阵，
这个范围是根据输入和输出维度计算出的。
PyTorch的`nn.init`模块提供了多种预置初始化方法。


### [**内置初始化**]

让我们首先调用内置的初始化器。
下面的代码将所有权重参数初始化为标准差为0.01的高斯随机变量，
且将偏置参数设置为0。


In [19]:
print(net)

Sequential(
  (0): Linear(in_features=4, out_features=8, bias=True)
  (1): ReLU()
  (2): Linear(in_features=8, out_features=1, bias=True)
)


In [ ]:
def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, mean=0, std=0.01)
        nn.init.zeros_(m.bias)
net.apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([-0.0214, -0.0015, -0.0100, -0.0058]), tensor(0.))

我们还可以将所有参数初始化为给定的常数，比如初始化为1。


In [20]:
def init_constant(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1)
        nn.init.zeros_(m.bias)
net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

我们还可以[**对某些块应用不同的初始化方法**]。
例如，下面我们使用Xavier初始化方法初始化第一个神经网络层，
然后将第三个神经网络层初始化为常量值42。


In [21]:
def init_xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)
def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42)
net[0].apply(init_xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([ 0.5410,  0.4284, -0.3442, -0.0786])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


### [**自定义初始化**]

有时，深度学习框架没有提供我们需要的初始化方法。
在下面的例子中，我们使用以下的分布为任意权重参数$w$定义初始化方法：

$$
\begin{aligned}
    w \sim \begin{cases}
        U(5, 10) & \text{ 可能性 } \frac{1}{4} \\
            0    & \text{ 可能性 } \frac{1}{2} \\
        U(-10, -5) & \text{ 可能性 } \frac{1}{4}
    \end{cases}
\end{aligned}
$$


同样，我们实现了一个`my_init`函数来应用到`net`。


In [ ]:
def my_init(m):
    if type(m) == nn.Linear:
        print("Init", *[(name, param.shape)
                        for name, param in m.named_parameters()][0])
        nn.init.uniform_(m.weight, -10, 10)
        m.weight.data *= m.weight.data.abs() >= 5
net.apply(my_init)
net[0].weight[:2]

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


tensor([[5.4079, 9.3334, 5.0616, 8.3095],
        [0.0000, 7.2788, -0.0000, -0.0000]], grad_fn=<SliceBackward0>)

In [23]:
print(net[0].weight)
print(net[0].weight+1)

Parameter containing:
tensor([[ 0.5410,  0.4284, -0.3442, -0.0786],
        [ 0.2242, -0.1344,  0.4235, -0.5624],
        [ 0.3753, -0.3780, -0.3231, -0.3150],
        [ 0.4927,  0.3213,  0.4483, -0.4920],
        [-0.3398,  0.0932, -0.5353, -0.3672],
        [-0.1785,  0.1128,  0.1760, -0.0135],
        [ 0.6592,  0.2519,  0.5926,  0.2301],
        [-0.0103,  0.1798,  0.0222,  0.2886]], requires_grad=True)
tensor([[1.5410, 1.4284, 0.6558, 0.9214],
        [1.2242, 0.8656, 1.4235, 0.4376],
        [1.3753, 0.6220, 0.6769, 0.6850],
        [1.4927, 1.3213, 1.4483, 0.5080],
        [0.6602, 1.0932, 0.4647, 0.6328],
        [0.8215, 1.1128, 1.1760, 0.9865],
        [1.6592, 1.2519, 1.5926, 1.2301],
        [0.9897, 1.1798, 1.0222, 1.2886]], grad_fn=<AddBackward0>)


In [28]:
net[0].weight.data[0,0]=42
print(net[0].weight.data)

tensor([[42.0000,  3.4284,  2.6558,  2.9214],
        [ 3.2242,  2.8656,  3.4235,  2.4376],
        [ 3.3753,  2.6220,  2.6769,  2.6850],
        [ 3.4927,  3.3213,  3.4483,  2.5080],
        [ 2.6602,  3.0932,  2.4647,  2.6328],
        [ 2.8215,  3.1128,  3.1760,  2.9865],
        [ 3.6592,  3.2519,  3.5926,  3.2301],
        [ 2.9897,  3.1798,  3.0222,  3.2886]])


注意，我们始终可以直接设置参数。


In [27]:
net[0].weight.data[:] += 1
net[0].weight.data[0, 0] = 42
net[0].weight.data[0]

tensor([42.0000,  3.4284,  2.6558,  2.9214])

## [**参数绑定**]

有时我们希望在多个层间共享参数：
我们可以定义一个稠密层，然后使用它的参数来设置另一个层的参数。


In [29]:
# 我们需要给共享层一个名称，以便可以引用它的参数
shared = nn.Linear(8, 8)
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                    shared, nn.ReLU(),
                    shared, nn.ReLU(),
                    nn.Linear(8, 1))

In [30]:
print(net)

Sequential(
  (0): Linear(in_features=4, out_features=8, bias=True)
  (1): ReLU()
  (2): Linear(in_features=8, out_features=8, bias=True)
  (3): ReLU()
  (4): Linear(in_features=8, out_features=8, bias=True)
  (5): ReLU()
  (6): Linear(in_features=8, out_features=1, bias=True)
)


In [31]:
net(X)
# 检查参数是否相同
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100
# 确保它们实际上是同一个对象，而不只是有相同的值
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])


这个例子表明第三个和第五个神经网络层的参数是绑定的。
它们不仅值相等，而且由相同的张量表示。
因此，如果我们改变其中一个参数，另一个参数也会改变。
这里有一个问题：当参数绑定时，梯度会发生什么情况？
答案是由于模型参数包含梯度，因此在反向传播期间第二个隐藏层
（即第三个神经网络层）和第三个隐藏层（即第五个神经网络层）的梯度会加在一起。


## 小结

* 我们有几种方法可以访问、初始化和绑定模型参数。
* 我们可以使用自定义初始化方法。

## 练习

1. 使用 :numref:`sec_model_construction` 中定义的`FancyMLP`模型，访问各个层的参数。
1. 查看初始化模块文档以了解不同的初始化方法。
1. 构建包含共享参数层的多层感知机并对其进行训练。在训练过程中，观察模型各层的参数和梯度。
1. 为什么共享参数是个好主意？


[Discussions](https://discuss.d2l.ai/t/1829)
